In [ ]:
import torchaudio
librispeech_test = torchaudio.datasets.LIBRISPEECH(".", url="test-clean", download=True)

100%|██████████| 331M/331M [00:12<00:00, 27.2MB/s]


In [ ]:
pip install fairseq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 43.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11291761 sha256=bdd696816486291ad12f85cb953e9c35aab0b1ea0750e05a06e88214b248df3d
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9502a0ac81b2448a1196159dacc32
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.wh

In [ ]:
import torch
import fairseq
import torchaudio

def load_vq_wav2vec_model(model_path):
    cp = torch.load(model_path, map_location=torch.device('cpu'))
    model, _, _ = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp])
    model = model[0]
    model.eval()
    return model

def preprocess_audio(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)
    # Resample if necessary
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)
    return waveform

def run_inference(model, audio_path):
    waveform = preprocess_audio(audio_path)
    z = model.feature_extractor(waveform)
    _, idxs = model.vector_quantizer.forward_idx(z)
    print(idxs.shape)  # output: torch.Size([1, 60, 2]), 60 timesteps with 2 indexes corresponding to 2 groups in the model

# Usage
model_path = '/content/vq-wav2vec.pt'
audio_path = '/content/LibriSpeech/test-clean/1089/134686/1089-134686-0000.flac'

model = load_vq_wav2vec_model(model_path)
run_inference(model, audio_path)

RuntimeError: unexpected EOF, expected 1625449 more bytes. The file might be corrupted.